In [80]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from PIL import ImageOps, Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt

In [2]:
TRAIN_DIR = './data/benign-malignant/train/'
TEST_DIR = './data/benign-malignant/test/'

CLASS = ['benign', 'malignant']

In [3]:
#Train 4 more models

#Pending
# 1. For melanoma and non-melanoma without any pre-processing  -> Data Required
# 2. For melanoma and non-melanoma with sharpening -> Data required
# 3. For benign and malignant with age and gender -> Data Found cleaning is required
# 4. For melanoma and non-melanoma without age and gender -> Data required
# 5. For melanoma and non-melanoma with "normalization" -> Data required


# 6. For benign and malignant with "normalization" -> Data Found Preprocessing is required
# 7. For bengin and malignant without preprocessing -> Data found -> Done
# 8. For benign and malignant with sharpening -> Data Found -> Done

In [8]:
# Normalizing images

# Model with processing
PRE_PROCESS = './data/benign-malignant/normalization/'

b_train = os.listdir(TRAIN_DIR + 'benign/')
b_test = os.listdir(TEST_DIR + 'benign/')

m_train = os.listdir(TRAIN_DIR + 'malignant/')
m_test = os.listdir(TEST_DIR + 'malignant/')

TRAIN_DIR = './data/benign-malignant/train/'
TEST_DIR = './data/benign-malignant/test/'

#Pre-processing training directory (NORMALIZATION)
#For benign
for i in range(len(b_train)):
    img = cv2.imread(TRAIN_DIR + 'benign/' + b_train[i])
    new_img = cv2.normalize(img, None, 
                            alpha=0, beta=200, 
                            norm_type=cv2.NORM_MINMAX)
    cv2.imwrite(PRE_PROCESS + 'train/benign/' + b_train[i], new_img)

#For malignant
for i in range(len(m_train)):
    img = cv2.imread(TRAIN_DIR + 'malignant/' + m_train[i])
    new_img = cv2.normalize(img, None, 
                            alpha=0, beta=200, 
                            norm_type=cv2.NORM_MINMAX)
    cv2.imwrite(PRE_PROCESS + 'train/malignant/' + m_train[i], new_img)


#Pre-processing validation directory
#For benign
for i in range(len(b_test)):
    img = cv2.imread(TEST_DIR + 'benign/' + b_test[i])
    new_img = cv2.normalize(img, None, 
                            alpha=0, beta=200, 
                            norm_type=cv2.NORM_MINMAX)
    cv2.imwrite(PRE_PROCESS + 'test/benign/' + b_test[i], new_img)

#For malignant
for i in range(len(m_test)):
    img = cv2.imread(TEST_DIR + 'malignant/' + m_test[i])
    new_img = cv2.normalize(img, None, 
                            alpha=0, beta=200, 
                            norm_type=cv2.NORM_MINMAX)
    cv2.imwrite(PRE_PROCESS + 'train/malignant/' + m_test[i], new_img)
    

generator = trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 50,
    width_shift_range = 2.0,
    height_shift_range = 2.0,    
    shear_range = 0.4,
    zoom_range = 2.0,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

TRAIN_DIR = PRE_PROCESS + 'train/'
TEST_DIR = PRE_PROCESS + 'test/'

train = generator.flow_from_directory(
    TRAIN_DIR,
    target_size = (225, 225),
    batch_size = 9,
    class_mode = 'binary'
)

val = generator.flow_from_directory(
    TEST_DIR,
    target_size = (225, 225),
    batch_size = 20,
    class_mode = 'binary'
)

model = tf.keras.models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
      train,
      epochs=20,
      validation_data=val
)

model.save('./models/with-normalization.h5')

Found 2937 images belonging to 2 classes.
Found 360 images belonging to 2 classes.
Epoch 1/20
327/327 [==============================] - 951s 3s/step - loss: 2.3331 - accuracy: 0.6902 - val_loss: 0.7121 - val_accuracy: 0.0000e+00
Epoch 2/20
327/327 [==============================] - 929s 3s/step - loss: 1.0744 - accuracy: 0.6823 - val_loss: 0.7030 - val_accuracy: 0.0000e+00
Epoch 3/20
327/327 [==============================] - 897s 3s/step - loss: 1.6647 - accuracy: 0.6881 - val_loss: 1.0007 - val_accuracy: 0.8111
Epoch 4/20
327/327 [==============================] - 867s 3s/step - loss: 0.9809 - accuracy: 0.7205 - val_loss: 1.1104 - val_accuracy: 0.4778
Epoch 5/20
327/327 [==============================] - 806s 2s/step - loss: 0.6073 - accuracy: 0.7637 - val_loss: 0.5338 - val_accuracy: 0.6944
Epoch 6/20
327/327 [==============================] - 818s 3s/step - loss: 0.5893 - accuracy: 0.7681 - val_loss: 0.9954 - val_accuracy: 0.4611
Epoch 7/20
327/327 [==============================]

In [95]:
TRAIN_DIR = './data/benign-malignant/train/'
TEST_DIR = './data/benign-malignant/test/'

generator = trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 50,
    width_shift_range = 2.0,
    height_shift_range = 2.0,    
    shear_range = 0.4,
    zoom_range = 2.0,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train = generator.flow_from_directory(
    TRAIN_DIR,
    target_size = (225, 225),
    batch_size = 9,
    class_mode = 'binary'
)

val = generator.flow_from_directory(
    TEST_DIR,
    target_size = (225, 225),
    batch_size = 20,
    class_mode = 'binary'
)

model = tf.keras.models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
      train,
      epochs=15,
      validation_data=val
)

model.save('./models/resent/without-preprocessing1.h5')

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.
Epoch 1/15
202/293 [===================>..........] - ETA: 3:44 - loss: 2.5164 - accuracy: 0.6810

In [ ]:
TRAIN_DIR = './data/benign-malignant/sharpened/'
TEST_DIR = './data/benign-malignant/sharpened/'

generator = trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 50,
    width_shift_range = 2.0,
    height_shift_range = 2.0,    
    shear_range = 0.4,
    zoom_range = 2.0,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train = generator.flow_from_directory(
    TRAIN_DIR,
    target_size = (225, 225),
    batch_size = 9,
    class_mode = 'binary'
)

val = generator.flow_from_directory(
    TEST_DIR,
    target_size = (225, 225),
    batch_size = 20,
    class_mode = 'binary'
)

model = tf.keras.models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
      train,
      epochs=15,
      validation_data=val
)

model.save('./models/resent/with-sharpening.h5')

In [ ]:
TRAIN_DIR = './data/benign-malignant/normalization/'
TEST_DIR = './data/benign-malignant/normalization/'

generator = trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 50,
    width_shift_range = 2.0,
    height_shift_range = 2.0,    
    shear_range = 0.4,
    zoom_range = 2.0,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

train = generator.flow_from_directory(
    TRAIN_DIR,
    target_size = (225, 225),
    batch_size = 9,
    class_mode = 'binary'
)

val = generator.flow_from_directory(
    TEST_DIR,
    target_size = (225, 225),
    batch_size = 20,
    class_mode = 'binary'
)

model = tf.keras.models.Sequential()
model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
      train,
      epochs=15,
      validation_data=val
)

model.save('./models/resent/with-normalization.h5')